# Group 1 Capstone Project Work
### Make sure to make / checkout your own branch before making changes

# Maddie Hince

In [10]:
import pandas as pd
import sqlite3

df = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df.head()


conn = sqlite3.connect('zippedData/im.db.zip')
pd.read_sql("""SELECT *""")

DatabaseError: Execution failed on sql '
SELECT *
FROM im
': file is not a database

# Cat Murad

In [2]:
import pandas as pd
import sqlite3

df_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
df_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [5]:
conn = sqlite3.connect('im.db')

In [30]:
pd.read_sql('select * from movie_akas', conn)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [13]:
pd.read_sql('select * from movie_basics', conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [70]:
q = pd.read_sql('''select distinct b.primary_title, b.genres, r.averagerating 
        from movie_basics b
        join movie_ratings r
            on b.movie_id = r.movie_id
        join movie_akas a
            on r.movie_id = a.movie_id
        where start_year >= 2010 and a.language is not Null and b.genres is not Null
        group by b.genres 
        order by averagerating DESC''', conn)
q

,primary_title,genres,averagerating
0,42nd Street: The Musical,Musical,9.1
1,My Champion,"Documentary,Sport",9.1
2,Twice Upon a Time,"Documentary,Family,War",9.0
3,The King,"Documentary,Family,Sport",8.9
4,Rock in the Red Zone,"Documentary,Music,War",8.8
...,...,...,...
622,Little Tigers,"Action,Comedy,War",2.8
623,The Brides of Sodom,"Fantasy,Horror,Sci-Fi",2.6
624,Vecherinka '1 noyabrya': Po Stranitsam detskih...,"Comedy,Fantasy,Sci-Fi",2.2
625,Khaltoor,"Action,Comedy,Musical",2.1


#### From above we can see that 
 - 13/20 films with the highest average ratings have the Documentary genre listed.
 - Other popular genres: war, sport, family, adventure, crime, thriller, drama

In [71]:
df = pd.DataFrame(q, columns = ['primary_title', 'genres', 'averagerating'])
print(df.head())

              primary_title                    genres  averagerating
0  42nd Street: The Musical                   Musical            9.1
1               My Champion         Documentary,Sport            9.1
2         Twice Upon a Time    Documentary,Family,War            9.0
3                  The King  Documentary,Family,Sport            8.9
4      Rock in the Red Zone     Documentary,Music,War            8.8


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627 entries, 0 to 626
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   primary_title  627 non-null    object 
 1   genres         627 non-null    object 
 2   averagerating  627 non-null    float64
dtypes: float64(1), object(2)
memory usage: 14.8+ KB


# Elijah Soba

# Luis Estrada 

# Kilmar Lazo 

# Michael Eugene